In [ ]:
!pip install pyreadstat

In [ ]:
!pip install scikit-posthocs

In [ ]:
pip install lifelines matplotlib

In [ ]:
!pip install nbconvert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pyreadstat
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

In [ ]:
dfe = pd.read_excel(r"C:\Users\jaime\Desktop\resultados TFG\Base_TFG_pruebas.xlsx")

In [ ]:
dfe.head()

In [ ]:
for columna in dfe.columns:
    num_faltantes = dfe[columna].isna().sum()
    
    print(f'Columna: {columna}')
    print(f'Número de valores faltantes: {num_faltantes}')
    print('---')

In [ ]:
dfe.loc[dfe['Sexo'] == 'Hombre', 'Sexo'] = 0
dfe.loc[dfe['Sexo'] == 'Mujer', 'Sexo'] = 1

In [ ]:
dfe.loc[dfe['tipo_procedimiento'] == 'ACTP', 'tipo_procedimiento'] = 0
dfe.loc[dfe['tipo_procedimiento'] == 'PONTAJE AORTOCORONARIO', 'tipo_procedimiento'] = 1

In [ ]:
dfe.loc[dfe['Tipo_intervención_mitral'] == 'Anuloplastia', 'Tipo_intervención_mitral'] = 'anuloplastia'

In [ ]:
columns_to_transform = ['HTA', 'DM', 'dislipemia', 'sobrepeso/obesidad', 'fumador', 'exfumador', 'IC', 'IAM', 'SC_intermedio/angina_inestable', 'Angina_estable', 'Antecedente_IAM']  # Especifica las columnas

In [ ]:
for col in columns_to_transform:
    dfe[col] = np.where(dfe[col] == 'SI', 1, 0)
dfe

In [ ]:
# Valores repetidos en Nhistoria
filas_repetidas = dfe[dfe["NHC"].duplicated(keep=False)]
filas_repetidas

In [ ]:
dfe = dfe.rename(columns={'sobrepeso/obesidad': 'sobrepeso_obesidad'})
dfe = dfe.rename(columns={'SC_intermedio/angina_inestable': 'SC_intermedio_Angina_inestable'})
dfe = dfe.rename(columns={' Intervención_mitral': 'Intervencion_mitral'})
dfe = dfe.rename(columns={'días_FEVI_pre': 'dias_FEVI_pre'})
dfe = dfe.rename(columns={'días_FEVI_post': 'dias_FEVI_post'})

In [ ]:
for columna in dfe.columns:
    num_faltantes = dfe[columna].isna().sum()
    
    print(f'Columna: {columna}')
    print(f'Número de valores faltantes: {num_faltantes}')
    print('---')

#### Crear variable Tabaquismo_Rec (No fumador = 0, Fumador = 1, Ex Fumador = 2)

In [ ]:
# Condiciones 'Tabaquismo_Rec'
conditions = [
    (dfe['fumador'] == 0) & (dfe['exfumador'] == 0),
    (dfe['fumador'] == 1) & (dfe['exfumador'] == 0),
    (dfe['fumador'] == 0) & (dfe['exfumador'] == 1) 
]

choices = [0, 1, 2]

dfe['Tabaquismo_Rec'] = np.select(conditions, choices, default=np.nan)

print("\nDataFrame con Tabaquismo_Rec:")
dfe.head()

#### Creamos la variable 'tipo_causa' donde agrupamos las categorías de la varibale 'Causa' en función de si son cardiovasculares o no

In [ ]:
categorias_unicas = dfe['Causa'].unique()
categorias_unicas

In [ ]:
causas_cardiovasculares = ['ICC ', 'ICC', 'IAM', 'Infarto agudo de miocarido', 'ACVA', 'muerte cardiovascular en residencia', 'Muerte  subita cardiovascular', 'Fallo agudo de trasplante cardiaco', 'CARDIOVASCULAR']

# Creamos variable 'tipo_causa'
dfe['tipo_causa'] = dfe['Causa'].apply(
    lambda x: 'Cardiovascular' if isinstance(x, str) and any(kw in x for kw in causas_cardiovasculares) 
    else ('No Cardiovascular' if isinstance(x, str) else np.nan)
)

print(dfe['tipo_causa'].value_counts(dropna=False))  # dropna=False para incluir los valores faltantes en el conteo

#### Creamos nuevas variables de pacientes que con lesión y que no han sido revascularizados 

In [ ]:
# Creamos nuevas variables de pacientes que con lesión y que no han sido revascularizados 
dfe['DA_sin_revasc'] = ((dfe['DA'] == 1) & (dfe['DA_revasc'] == 0)).astype(int)
dfe['Cx_sin_revasc'] = ((dfe['Cx'] == 1) & (dfe['Cx_revasc'] == 0)).astype(int)
dfe['CD_sin_revasc'] = ((dfe['CD'] == 1) & (dfe['CD_revasc'] == 0)).astype(int)

#### Creamos la variable 'Muerte_o_Infarto' en función de si el paciente ha fallecido o ha sufrido un infarto en los primeros tres años tras la intervención

In [ ]:
# Definir la nueva columna 'Muerte_o_Infarto'
def calcular_muerte_infarto(row):
    limite_dias = 3 * 365

    # Caso 1: Muerte=0, Infarto=0
    if row['Muerte'] == 0 and row['Infarto'] == 0:
        return 0

    # Caso 2: Muerte=1, Infarto=1
    if row['Muerte'] == 1 and row['Infarto'] == 1:
        diferencia_muerte = (row['Fecha_muerte'] - row['Fecha_Intervención']).days if pd.notna(row['Fecha_muerte']) else float('inf')
        diferencia_infarto = (row['Fecha_infarto'] - row['Fecha_Intervención']).days if pd.notna(row['Fecha_infarto']) else float('inf')

        if diferencia_muerte <= limite_dias or diferencia_infarto <= limite_dias:
            return 1
        else:
            return 0

    # Caso 3: Muerte=1, Infarto=0
    if row['Muerte'] == 1 and row['Infarto'] == 0:
        diferencia_muerte = (row['Fecha_muerte'] - row['Fecha_Intervención']).days if pd.notna(row['Fecha_muerte']) else float('inf')

        if diferencia_muerte <= limite_dias:
            return 1
        else:
            return 0

    # Caso 4: Muerte=0, Infarto=1
    if row['Muerte'] == 0 and row['Infarto'] == 1:
        diferencia_infarto = (row['Fecha_infarto'] - row['Fecha_Intervención']).days if pd.notna(row['Fecha_infarto']) else float('inf')

        if diferencia_infarto <= limite_dias:
            return 1
        else:
            return 0

# Aplicar la función a cada fila del DataFrame
dfe['Muerte_o_Infarto'] = dfe.apply(calcular_muerte_infarto, axis=1)

dfe[['Fecha_Intervención', 'Muerte', 'Fecha_muerte', 'Infarto', 'Fecha_infarto', 'Muerte_o_Infarto']]

#### Graficamos variables categóricas y realizamos conteo de cada una de las categorías de cada variable

In [ ]:
var_cat = ['Sexo', 'Muerte', 'Causa', 'tipo_causa', 'Infarto', 'Nueva_revascularización', 'Ingeso_ICC', 'Tronco_coronario', 'DA', 'Cx', 'CD', 
           'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Revas_completa', 'dif', 'Vasosrev', 'Vasospre', 'Intervencion_mitral', 'Tipo_intervención_mitral', 
           'tipo_procedimiento', 'HTA', 'DM', 'dislipemia', 'sobrepeso_obesidad', 'fumador', 'exfumador', 'Tabaquismo_Rec', 'IC', 'IAM', 
           'SC_intermedio_Angina_inestable', 'Angina_estable', 'Antecedente_IAM', 'Muerte_o_Infarto', 'DA_sin_revasc', 'Cx_sin_revasc', 'CD_sin_revasc'] 
# Crear un gráfico para cada variable categórica
for var in var_cat:
    plt.figure(figsize=(8, 5))
    sns.countplot(data=dfe, x=var)
    
    # Agregar los conteos sobre las barras
    ax = plt.gca()
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

    # Título y etiquetas
    plt.title(f'Conteo de categorías en {var}', fontsize=16)
    plt.xlabel(var, fontsize=14)
    plt.ylabel('Conteo', fontsize=14)
    
    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()

In [ ]:
# Crear una lista para almacenar los resultados
summary_table = []

# Iterar sobre cada variable categórica
for var in var_cat:
    # Obtener los conteos de cada categoría
    counts = dfe[var].value_counts(dropna=False)  # Incluir NaN si existe
    percentages = dfe[var].value_counts(normalize=True, dropna=False) * 100  # Porcentajes
    
    # Crear una tabla por cada categoría de la variable
    for category, count in counts.items():
        percentage = percentages[category]
        summary_table.append({
            'Variable': var,
            'Categoría': category,
            'Conteo': count,
            'Porcentaje (%)': round(percentage, 2)
        })

# Convertir la lista a un DataFrame
df_summary = pd.DataFrame(summary_table)

# Mostrar la tabla
df_summary

#### Creación variable 'tiempo_meses' (meses transcurridos entre la fecha de intervención y la de muerte)

In [ ]:
# Creamos columna para la fecha de fin de seguimiento (última fecha conocida)
# Si no tienes una fecha límite clara, podrías usar una fecha fija
fecha_fin_seguimiento = pd.Timestamp("2024-11-20")  # Ajusta según sea necesario

# Calcular la diferencia en meses entre las fechas
dfe['tiempo_meses'] = (
    dfe['Fecha_muerte'].fillna(fecha_fin_seguimiento).dt.to_period('M') - 
    dfe['Fecha_Intervención'].dt.to_period('M')
).apply(lambda x: x.n)  # Convertir PeriodIndex a valores numéricos


# Mostrar una vista previa
dfe[['Fecha_Intervención', 'Fecha_muerte', 'tiempo_meses', 'Muerte']]

#### Graficamos variables numéricas 

In [ ]:
var_num = ['edad_ingreso', 'dias_muerte', 'FEVI_pre', 'dias_FEVI_pre', 'FEVI_post', 'dias_FEVI_post', 'tiempo_meses']
# Crear un histograma para cada variable numérica
for var in var_num:
    plt.figure(figsize=(8, 5))
    dfe[var].hist(bins=10, edgecolor='black')  
    
    # Título y etiquetas
    plt.title(f'Histograma de {var}', fontsize=16)
    plt.xlabel(var, fontsize=14)
    plt.ylabel('Frecuencia', fontsize=14)
    
    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()

In [ ]:
# Bucle para generar los histogramas
for col in var_num:
    plt.figure(figsize=(10, 6))  # Ajustar tamaño de la figura para mejor visibilidad
    
    # Generar el histograma con más detalles ajustando el número de bins
    plt.hist(dfe[col].dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7) 
    
    # Personalización del gráfico
    plt.title(f'Histograma de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.grid(True)
    
    # Mostrar el gráfico
    plt.show()

#### Estadística básica de variables numéricas

In [ ]:
# Lista para almacenar la información de cada variable
summary_data = []

# Bucle para recorrer cada columna de tipo float64
for col in var_num:
    # Calcular estadísticas básicas
    media = dfe[col].mean()  
    mediana = dfe[col].median()  
    desviacion_std = dfe[col].std()  
    minimo = dfe[col].min() 
    maximo = dfe[col].max()  
    
    # Número de valores faltantes (NaN) en la columna
    num_faltantes = dfe[col].isna().sum()  # Solo cuenta los NaN, no los 0

    # Identificar outliers usando el método del rango intercuartílico (IQR)
    Q1 = dfe[col].quantile(0.25)  # Primer cuartil
    Q3 = dfe[col].quantile(0.75)  # Tercer cuartil
    IQR = Q3 - Q1  # Rango intercuartílico
    
    # Definir límites para los outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrar los valores que son outliers
    outliers = dfe[col][(dfe[col] < lower_bound) | (dfe[col] > upper_bound)]
    
    # Contar cuántos outliers hay
    num_outliers = outliers.shape[0]
    
    # Rango de los outliers (si existen)
    if num_outliers > 0:
        outlier_min = outliers.min()
        outlier_max = outliers.max()
    else:
        outlier_min, outlier_max = np.nan, np.nan  # Si no hay outliers, asignar NaN

    # Agregar la información a la lista
    summary_data.append({
        'Variable': col,
        'Valores Faltantes': num_faltantes,
        'Media': media,
        'Mediana': mediana,
        'Desviación Estándar': desviacion_std,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Mínimo': minimo,
        'Máximo': maximo,
        'Número de Outliers': num_outliers,
        'Rango de Outliers (Mín)': outlier_min,
        'Rango de Outliers (Máx)': outlier_max
    })

# Crear un DataFrame con el resumen
df_summary = pd.DataFrame(summary_data)

# Mostrar el DataFrame con el resumen
df_summary

#### Comparación variables entre pacientes con y sin evento

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, shapiro

# Separamos grupos (evento y no evento)
grupo_evento = dfe[dfe['Muerte'] == 1]  # Pacientes con evento
grupo_no_evento = dfe[dfe['Muerte'] == 0]  # Pacientes sin evento

# Almacenar resultados
resultados_categoricas = []
resultados_numericas = []
tablas_contingencia = []
frecuencias_categoricas = []

# Comparación variables categóricas
var_cat = ['Sexo', 'Causa', 'tipo_causa', 'Infarto', 'Nueva_revascularización', 'Ingeso_ICC', 'Tronco_coronario', 'DA', 'Cx', 'CD', 
           'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Revas_completa', 'dif', 'Vasosrev', 'Vasospre', ' Intervencion_mitral', 'Tipo_intervención_mitral', 
           'tipo_procedimiento', 'HTA', 'DM', 'dislipemia', 'sobrepeso_obesidad', 'fumador', 'exfumador', 'Tabaquismo_Rec', 'IC', 'IAM', 
           'SC_intermedio_Angina_inestable', 'Angina_estable', 'Antecedente_IAM', 'Muerte_o_Infarto', 'DA_sin_revasc', 'Cx_sin_revasc', 'CD_sin_revasc']

for var in var_cat:
    if var in dfe.columns:
        tabla = pd.crosstab(dfe[var], dfe['Muerte'])  # Tabla de contingencia
        chi2, p, _, _ = chi2_contingency(tabla)  # Prueba de chi-cuadrado
        
        # Guardar tabla de contingencia como DataFrame para exportar a Excel
        tabla_reset = tabla.reset_index()
        tabla_reset.insert(0, 'Variable', var)  # Agregar nombre de variable
        tablas_contingencia.append(tabla_reset)
        
        # Guardar frecuencias individuales
        tabla_frec = dfe[var].value_counts().reset_index()
        tabla_frec.columns = ['Valor', 'Frecuencia']
        tabla_frec.insert(0, 'Variable', var)
        frecuencias_categoricas.append(tabla_frec)
        
        # Guardar resultado de la prueba
        resultados_categoricas.append({'Variable': var, 'Prueba': 'Chi-cuadrado', 'P-valor': p})

# Comparación variables numéricas
var_num = ['edad_ingreso', 'dias_muerte', 'FEVI_pre', 'dias_FEVI_pre', 'FEVI_post', 'dias_FEVI_post', 'tiempo_meses']

for var in var_num:
    if var in dfe.columns:
        # Evaluar normalidad con Shapiro-Wilk
        stat, p_normal = shapiro(dfe[var].dropna())  
        
        if p_normal > 0.05:  # Distribución normal → Prueba t de Student
            test_name = 'Prueba t de Student'
            t_stat, p_value = ttest_ind(grupo_evento[var].dropna(), grupo_no_evento[var].dropna(), equal_var=False)
        else:  # No normal → Prueba de Mann-Whitney U
            test_name = 'Mann-Whitney U'
            u_stat, p_value = mannwhitneyu(grupo_evento[var].dropna(), grupo_no_evento[var].dropna())
        
        # Guardar resultados
        resultados_numericas.append({'Variable': var, 'Prueba': test_name, 'P-valor': p_value, 'P-valor Shapiro-Wilk': p_normal})

# Convertir a DataFrame
df_resultados_categoricas = pd.DataFrame(resultados_categoricas).sort_values(by='P-valor')
df_resultados_numericas = pd.DataFrame(resultados_numericas).sort_values(by='P-valor')
df_tablas_contingencia = pd.concat(tablas_contingencia, ignore_index=True) if tablas_contingencia else pd.DataFrame()
df_frecuencias_categoricas = pd.concat(frecuencias_categoricas, ignore_index=True) if frecuencias_categoricas else pd.DataFrame()

In [ ]:
# Guardar en un archivo Excel con múltiples hojas
nombre_archivo = "comparacion_muerte_variables.xlsx"
with pd.ExcelWriter(nombre_archivo) as writer:
    df_resultados_categoricas.to_excel(writer, sheet_name="Variables_Categoricas", index=False)
    df_resultados_numericas.to_excel(writer, sheet_name="Variables_Numericas", index=False)
    if not df_tablas_contingencia.empty:
        df_tablas_contingencia.to_excel(writer, sheet_name="Tablas_Contingencia", index=False)
    if not df_frecuencias_categoricas.empty:
        df_frecuencias_categoricas.to_excel(writer, sheet_name="Frecuencias_Categoricas", index=False)

print(f"Resultados guardados en {nombre_archivo}")

#### Comparación variables entre pacientes con y sin evento

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, shapiro

# Separamos grupos (evento y no evento)
grupo_evento = dfe[dfe['Muerte'] == 1]  # Pacientes con evento
grupo_no_evento = dfe[dfe['Muerte'] == 0]  # Pacientes sin evento

# Almacenamos resultados
resultados_categoricas = []
resultados_numericas = []

# Comparamos variables categóricas
var_cat = ['Sexo', 'Causa', 'tipo_causa', 'Infarto', 'Nueva_revascularización', 'Ingeso_ICC', 'Tronco_coronario', 'DA', 'Cx', 'CD', 
           'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Revas_completa', 'dif', 'Vasosrev', 'Vasospre', ' Intervencion_mitral', 'Tipo_intervención_mitral', 
           'tipo_procedimiento', 'HTA', 'DM', 'dislipemia', 'sobrepeso_obesidad', 'fumador', 'exfumador', 'Tabaquismo_Rec', 'IC', 'IAM', 
           'SC_intermedio_Angina_inestable', 'Angina_estable', 'Antecedente_IAM', 'Muerte_o_Infarto', 'DA_sin_revasc', 'Cx_sin_revasc', 'CD_sin_revasc']

for var in var_cat:
    if var in dfe.columns:
        tabla = pd.crosstab(dfe[var], dfe['Muerte'])  # Tabla de contingencia
        chi2, p, _, _ = chi2_contingency(tabla)  # Prueba de chi-cuadrado
        
        # Guardar resultado de la prueba
        resultados_categoricas.append({'Variable': var, 'P-valor': p})

# Comparamos variables numéricas
var_num = ['edad_ingreso', 'dias_muerte', 'FEVI_pre', 'dias_FEVI_pre', 'FEVI_post', 'dias_FEVI_post', 'tiempo_meses']

for var in var_num:
    if var in dfe.columns:
        # Shapiro-Wilk (normalidad)
        stat, p_normal = shapiro(dfe[var].dropna())  
        
        if p_normal > 0.05:  # Distr. normal (t-test)
            test_name = 'Prueba t de Student'
            t_stat, p_value = ttest_ind(grupo_evento[var].dropna(), grupo_no_evento[var].dropna(), equal_var=False)
        else:  # Distr. NO normal (Mann-Whitney U)
            test_name = 'Mann-Whitney U'
            u_stat, p_value = mannwhitneyu(grupo_evento[var].dropna(), grupo_no_evento[var].dropna())
        
        # Guardamos resultados
        resultados_numericas.append({'Variable': var, 'Prueba': test_name, 'P-valor': p_value, 'P-valor Shapiro-Wilk': p_normal})

df_resultados_categoricas = pd.DataFrame(resultados_categoricas)
df_resultados_numericas = pd.DataFrame(resultados_numericas)

In [ ]:
# Guardar en un archivo Excel con múltiples hojas
nombre_archivo = "comparacion_est_variables.xlsx"
with pd.ExcelWriter(nombre_archivo) as writer:
    df_resultados_categoricas.to_excel(writer, sheet_name="Variables_Categoricas", index=False)
    df_resultados_numericas.to_excel(writer, sheet_name="Variables_Numericas", index=False)

print(f"Resultados guardados en {nombre_archivo}")

##### Tablas de contingencia y de proporciones (variables categóricas)

In [ ]:
for var in var_cat:
    if var in dfe.columns:
        # Tabla de contingencia
        tabla_contingencia = pd.crosstab(dfe[var], dfe['Muerte'])
        print(f"\n Tabla de Contingencia - {var}:")
        print(tabla_contingencia)

        # Tabla de proporciones
        tabla_proporciones = pd.crosstab(dfe[var], dfe['Muerte'], normalize='columns')
        print(f"\n Tabla de Proporciones - {var}:")
        print(tabla_proporciones)

#### Comparación de variables respecto a Muerte_o_Infarto

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact, ttest_ind, mannwhitneyu, shapiro

# Separamos grupos (evento y no evento)
grupo_evento = dfe[dfe['Muerte_o_Infarto'] == 1]  # Pacientes con evento
grupo_no_evento = dfe[dfe['Muerte_o_Infarto'] == 0]  # Pacientes sin evento

# Almacenamos resultados
resultados_categoricas = []
resultados_numericas = []

# Comparamos variables categóricas
var_cat = ['Sexo', 'Nueva_revascularización', 'Ingeso_ICC', 'Tronco_coronario', 'DA', 'Cx', 'CD', 'Revas_completa', 'dif', 'Vasosrev', 'Vasospre', 
           'Intervencion_mitral', 'tipo_procedimiento', 'HTA', 'DM', 'dislipemia', 'sobrepeso_obesidad', 'fumador', 'exfumador', 'IC', 'IAM', 
           'SC_intermedio_Angina_inestable', 'Angina_estable', 'Antecedente_IAM', 'DA_sin_revasc', 'Cx_sin_revasc', 'CD_sin_revasc']

for var in var_cat:
    if var in dfe.columns:
        tabla = pd.crosstab(dfe[var], dfe['Muerte_o_Infarto'])  # Tabla de contingencia
        chi2, p_chi2, _, expected = chi2_contingency(tabla)  # Prueba de chi-cuadrado con frecuencias esperadas
        
        # Revisar si alguna celda tiene <5 casos
        if (expected < 5).any():
            # Si alguna frecuencia esperada es menor a 5, usamos Fisher
            if tabla.shape == (2, 2):  # Fisher solo para tablas 2x2
                _, p_fisher = fisher_exact(tabla)
                p_value = p_fisher
                test_name = 'Fisher Exact'
            else:
                p_value = p_chi2  # Para tablas más grandes, mantenemos Chi-cuadrado
                test_name = 'Chi-cuadrado (ajustado)'
        else:
            p_value = p_chi2
            test_name = 'Chi-cuadrado'

        # Guardar resultado de la prueba
        resultados_categoricas.append({'Variable': var, 'Prueba': test_name, 'P-valor': p_value})

# Comparamos variables numéricas
var_num = ['edad_ingreso', 'FEVI_pre', 'dias_FEVI_pre', 'FEVI_post', 'dias_FEVI_post']

for var in var_num:
    if var in dfe.columns:
        # Shapiro-Wilk (normalidad)
        stat, p_normal = shapiro(dfe[var].dropna())  
        
        if p_normal > 0.05:  # Distr. normal (t-test)
            test_name = 'Prueba t de Student'
            t_stat, p_value = ttest_ind(grupo_evento[var].dropna(), grupo_no_evento[var].dropna(), equal_var=False)
        else:  # Distr. NO normal (Mann-Whitney U)
            test_name = 'Mann-Whitney U'
            u_stat, p_value = mannwhitneyu(grupo_evento[var].dropna(), grupo_no_evento[var].dropna())
        
        # Guardamos resultados
        resultados_numericas.append({'Variable': var, 'Prueba': test_name, 'P-valor': p_value, 'P-valor Shapiro-Wilk': p_normal})

df_resultados_categoricas = pd.DataFrame(resultados_categoricas)
df_resultados_numericas = pd.DataFrame(resultados_numericas)

In [ ]:
# Guardar en un archivo Excel con múltiples hojas
nombre_archivo = "comparacion_muerteoinfarto.xlsx"
with pd.ExcelWriter(nombre_archivo) as writer:
    df_resultados_categoricas.to_excel(writer, sheet_name="Variables_Categoricas", index=False)
    df_resultados_numericas.to_excel(writer, sheet_name="Variables_Numericas", index=False)

print(f"Resultados guardados en {nombre_archivo}")

In [ ]:
for var in var_cat:
    if var in dfe.columns:
        # Tabla de contingencia
        tabla_contingencia = pd.crosstab(dfe[var], dfe['Muerte_o_Infarto'])
        print(f"\n Tabla de Contingencia - {var}:")
        print(tabla_contingencia)

        # Tabla de proporciones
        tabla_proporciones = pd.crosstab(dfe[var], dfe['Muerte_o_Infarto'], normalize='columns')
        print(f"\n Tabla de Proporciones - {var}:")
        print(tabla_proporciones)

###### Distr. proporciones variables categóricas

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

for var in var_cat:
    # Calculamos proporciones de cada categoría dentro de los grupos de 'muerte_o_infarto'
    prop_dfe = pd.crosstab(dfe[var], dfe['Muerte_o_Infarto'], normalize='columns')
    
    # Graficamos
    prop_dfe.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set2')
    plt.title(f'Distribución Proporcional de {var} entre Pacientes con y sin Evento', fontsize=14)
    plt.xlabel(var, fontsize=12)
    plt.ylabel('Proporción', fontsize=12)
    plt.xticks(rotation=0)  # Rotar las etiquetas en el eje X si es necesario
    plt.legend(title='Muerte o Infarto', loc='upper right', labels=['Sin Evento', 'Con Evento'])
    plt.show()

###### Distr. variables numéricas

In [ ]:
for var in var_num:
    plt.figure(figsize=(8, 6))
    sns.histplot(dfe[dfe['Muerte_o_Infarto'] == 0][var], kde=True, color="blue", label="Sin Evento", alpha=0.5)
    sns.histplot(dfe[dfe['Muerte_o_Infarto'] == 1][var], kde=True, color="red", label="Con Evento", alpha=0.5)
    plt.title(f'Distribución de {var} entre Pacientes con y sin Evento', fontsize=14)
    plt.xlabel(var, fontsize=12)
    plt.ylabel('Frecuencia', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

### Análisis de Supervivencia 

#### Kaplan-Meier

In [ ]:
grupo1 = dfe[dfe['FEVI_pre'] <= 40]  # Grupo con FEVI_pre <= 40
grupo2 = dfe[dfe['FEVI_pre'] > 40]  # Grupo con FEVI_pre > 40

kmf = KaplanMeierFitter()

plt.figure(figsize=(10, 6))

# Grupo 1
kmf.fit(durations=grupo1['tiempo_meses'], event_observed=grupo1['Muerte'])
kmf.plot_survival_function(label='FEVI_pre <= 40', color='red')

# Grupo 2
kmf.fit(durations=grupo2['tiempo_meses'], event_observed=grupo2['Muerte'])
kmf.plot_survival_function(label='FEVI_pre > 40', color='blue')

plt.title('Curva de Supervivencia (Kaplan-Meier)')
plt.xlabel('tiempo_meses')
plt.ylabel('Probabilidad de Supervivencia')
plt.legend()
plt.grid(True)
max_time = max(grupo1['tiempo_meses'].max(), grupo2['tiempo_meses'].max())  # Máximo valor de tiempo
ticks = np.arange(0, max_time + 12, 12)  # Intervalos de 12 en 12 meses
plt.xticks(ticks, labels=[str(int(tick)) for tick in ticks])
plt.show()

#### log-rank test

In [ ]:
from lifelines.statistics import logrank_test

# log-rank test
result = logrank_test(
    grupo1['tiempo_meses'], grupo2['tiempo_meses'],  # Tiempos de supervivencia de ambos grupos
    event_observed_A=grupo1['Muerte'],             # Eventos (muerte) para el grupo 1
    event_observed_B=grupo2['Muerte']              # Eventos (muerte) para el grupo 2
)

print("Estadístico de la prueba:", result.test_statistic)
print("p-valor:", result.p_value)

if result.p_value < 0.05:
    print("Existen diferencias significativas entre las curvas de supervivencia.")
else:
    print("No se encontraron diferencias significativas entre las curvas de supervivencia.")

#### Regresión logística con variables escaladas y umbral optimo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
)
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte', 
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División train-test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Modelo
logreg = LogisticRegression(solver='liblinear', max_iter=1000)
logreg.fit(X_train, y_train)

# Predicción probabilística
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

# Índice de Youden para umbral óptimo
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
youden_index = tpr - fpr
best_idx = np.argmax(youden_index)
best_threshold = thresholds[best_idx]
y_pred_adj = (y_pred_proba >= best_threshold).astype(int)

# Métricas
cm = confusion_matrix(y_test, y_pred_adj)
tn, fp, fn, tp = cm.ravel()
sensibilidad = tp / (tp + fn)
especificidad = tn / (tn + fp)
accuracy = accuracy_score(y_test, y_pred_adj)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Salidas
print(f"\nMejor umbral según índice de Youden: {best_threshold:.2f}")
print("\nMatriz de Confusión (con umbral óptimo):")
print(cm)
print(f"\nAccuracy: {accuracy:.2f}")
print(f"Sensibilidad (Recall): {sensibilidad:.2f}")
print(f"Especificidad: {especificidad:.2f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_adj))

print(f"\nROC-AUC Score: {roc_auc:.2f}")

# Visualización: Matriz de Confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Evento', 'Evento'], yticklabels=['No Evento', 'Evento'])
plt.title('Matriz de Confusión (Umbral Óptimo)')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

# Curva ROC con punto óptimo
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='red')
plt.scatter(fpr[best_idx], tpr[best_idx], color='black', label=f'Umbral óptimo (Youden = {best_threshold:.2f})', zorder=5)
plt.title('Curva ROC')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

# Convertir X_test a DataFrame con nombres originales de columnas
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Explicador SHAP para modelos lineales
explainer = shap.Explainer(logreg, X_test_df)

# Obtener valores SHAP
shap_values = explainer(X_test_df)

# Extraer valores SHAP en matriz (como en Random Forest)
shap_array = shap_values.values  # array de tamaño (n_muestras, n_features)

# Graficar SHAP Summary Plot con todas las variables
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_array, X_test_df, plot_type="dot", max_display=len(X_test_df.columns))

In [ ]:
# Obtener nombres originales de las variables del DataFrame original
feature_names = X.columns

# Extraer coeficientes del modelo ya entrenado
coefs = logreg.coef_[0]

# Crear un DataFrame con nombres y coeficientes
importancia_coef = pd.DataFrame({
    'Variable': feature_names,
    'Coeficiente': coefs,
    'Importancia_absoluta': np.abs(coefs)
})

# Ordenar por importancia (valor absoluto del coeficiente)
importancia_coef = importancia_coef.sort_values(by='Importancia_absoluta', ascending=False)

# Mostrar tabla ordenada
print("\nCoeficientes del modelo de regresión logística ordenados por importancia:")
print(importancia_coef[['Variable', 'Coeficiente']].to_string(index=False))

#### Regresión Logística configurada con validacion cruzada y umbral optimo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_curve, auc, confusion_matrix, accuracy_score
)

# Preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto',
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte',
           'Infarto', 'dias_muerte', 'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc',
           'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

tprs = []
aucs = []
youden_thresholds = []
youden_points = []
mean_fpr = np.linspace(0, 1, 100)
metricas_fold = []

# Acumuladores para matriz de confusión total
y_true_total = []
y_pred_total = []

plt.figure(figsize=(10, 8))

for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = LogisticRegression(solver='liblinear', max_iter=1000)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    roc_auc = auc(fpr, tpr)

    # Índice de Youden
    youden_index = tpr - fpr
    best_idx = np.argmax(youden_index)
    best_threshold = thresholds[best_idx]
    youden_thresholds.append(best_threshold)
    youden_points.append((fpr[best_idx], tpr[best_idx]))

    # Predicción binaria con umbral óptimo
    y_pred = (y_prob >= best_threshold).astype(int)

    # Guardar predicciones
    y_true_total.extend(y_test)
    y_pred_total.extend(y_pred)

    # Métricas
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensibilidad = tp / (tp + fn)
    especificidad = tn / (tn + fp)
    accuracy = accuracy_score(y_test, y_pred)

    metricas_fold.append({
        'Fold': fold_idx,
        'AUC': roc_auc,
        'Sensibilidad': sensibilidad,
        'Especificidad': especificidad,
        'Accuracy': accuracy,
        'Umbral óptimo (Youden)': best_threshold
    })

    # Curva ROC
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1.5, alpha=0.6,
             label=f'Fold {fold_idx} (AUC = {roc_auc:.2f}, Youden = {best_threshold:.2f})')

# Media de curvas ROC
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
mean_threshold = np.mean(youden_thresholds)

plt.plot(mean_fpr, mean_tpr, color='black',
         label=f'Media (AUC = {mean_auc:.2f}, Youden = {mean_threshold:.2f})',
         lw=2.5, linestyle='--')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=1)
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curvas ROC por Fold y Media\nRegresión Logística con Validación Cruzada (Youden)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

# Tabla de métricas
df_metricas = pd.DataFrame(metricas_fold)
print("\nMétricas por Fold:")
print(df_metricas)

# Promedio de métricas
print("\nPromedio de métricas:")
print(df_metricas.drop(columns='Fold').mean())

# Matriz de Confusión Total
cm_total = confusion_matrix(y_true_total, y_pred_total)
print("\nMatriz de Confusión Total:")
print(cm_total)

# Mostrar matriz de confusión total como heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_total, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Evento', 'Evento'], yticklabels=['No Evento', 'Evento'])
plt.title('Matriz de Confusión Total\nValidación Cruzada (Umbral Youden por Fold)')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.tight_layout()
plt.show()

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto',
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte',
           'Infarto', 'dias_muerte', 'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc',
           'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_df_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Entrenar modelo sobre todos los datos
logreg = LogisticRegression(solver='liblinear', max_iter=1000)
logreg.fit(X_df_scaled, y)

# Crear un masker para datos independientes (nuevo enfoque recomendado)
masker = shap.maskers.Independent(data=X_df_scaled)

# Crear el explicador SHAP sin el parámetro obsoleto
explainer = shap.Explainer(logreg, masker)

# Obtener valores SHAP
shap_values = explainer(X_df_scaled)

# Graficar todos los valores SHAP
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values.values, X_df_scaled, plot_type="dot", max_display=len(X.columns))

In [ ]:
logreg.coef_

In [ ]:
X.columns

In [ ]:
# Inspeccionar las columnas de X después de la exclusión
print("Columnas seleccionadas:", X.columns.tolist())

# Verificar tipos de datos
print("Tipos de datos en X:")
print(X.dtypes.value_counts())

In [ ]:
# Revisar valores únicos en cada columna
for col in X.columns:
    print(f"Columna: {col}, Tipos: {X[col].map(type).unique()}")

#### Validacion cruzada datos escalados + Youden

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, roc_curve
)
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar y preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte',
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []
overall_confusion_matrix = np.zeros((2, 2))

# Ejecutar validación cruzada
for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Modelo
    logreg = LogisticRegression(solver='liblinear', max_iter=1000)
    logreg.fit(X_train, y_train)

    # Predicciones probabilísticas
    y_pred_proba = logreg.predict_proba(X_test)[:, 1]

    # Calcular índice de Youden para encontrar el mejor umbral
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    youden_index = tpr - fpr
    best_idx = np.argmax(youden_index)
    best_threshold = thresholds[best_idx]

    # Clasificación con umbral óptimo
    y_pred = (y_pred_proba >= best_threshold).astype(int)

    # Métricas
    cm = confusion_matrix(y_test, y_pred)
    overall_confusion_matrix += cm
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    print(f"Fold {fold}: Mejor umbral (Youden) = {best_threshold:.2f}")
    print(f"Fold {fold}: Sensibilidad = {recall:.2f}, Especificidad = {cm[0,0]/(cm[0,0]+cm[0,1]):.2f}, Accuracy = {accuracy:.2f}\n")

    fold_results.append({
        'Fold': fold,
        'Threshold': best_threshold,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1 Score': f1,
        'ROC-AUC': roc_auc
    })

# Resultados
results_df = pd.DataFrame(fold_results)
print("Resultados por Fold:")
print(results_df)

# Promedios
mean_results = results_df.mean(numeric_only=True)
print("\nPromedio de métricas:")
print(mean_results)

# Matriz de confusión total
print("\nMatriz de Confusión Total:")
print(overall_confusion_matrix)

# Visualización de métricas por fold
results_df[['Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC-AUC']].boxplot()
plt.title('Distribución de métricas por Fold (Umbral óptimo Youden)')
plt.ylabel('Score')
plt.grid(alpha=0.3)
plt.show()

#### Curvas ROC folds validacion cruzada + youden

In [ ]:
from matplotlib.colors import to_rgba
from matplotlib import colormaps
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Cargar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte',
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Función para aclarar colores
def lighten_color(color, amount=0.5):
    r, g, b, a = to_rgba(color)
    return (r + (1 - r) * amount, g + (1 - g) * amount, b + (1 - b) * amount, a)

# Preparar colores
colors = colormaps['tab10']
colors_light = [lighten_color(colors(i / 9), amount=0.5) for i in range(5)]

# Variables para curva promedio
tprs = []
mean_fpr = np.linspace(0, 1, 100)

# Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Graficar curvas ROC por fold
plt.figure(figsize=(10, 6))
for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Entrenar modelo
    logreg = LogisticRegression(solver='liblinear', max_iter=1000)
    logreg.fit(X_train, y_train)
    y_pred_proba = logreg.predict_proba(X_test)[:, 1]

    # Curva ROC y Youden
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    youden_index = tpr - fpr
    best_idx = np.argmax(youden_index)
    best_threshold = thresholds[best_idx]
    best_fpr = fpr[best_idx]
    best_tpr = tpr[best_idx]

    # Interpolación para curva media
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Dibujar curva ROC del fold
    plt.plot(fpr, tpr, color=colors_light[fold - 1],
             label=f'Fold {fold} (AUC = {roc_auc:.2f}, Threshold = {best_threshold:.2f})')
    
    # Punto óptimo
    plt.scatter(best_fpr, best_tpr, color=colors_light[fold - 1], edgecolors='black', zorder=5)

# Curva ROC media
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue', linewidth=2, label=f'Mean ROC (AUC = {mean_auc:.2f})')

# Línea de referencia
plt.plot([0, 1], [0, 1], linestyle='--', color='red')

# Estética
plt.title('Curvas ROC por Fold con Umbral Óptimo (Youden)', fontsize=16)
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### Extraemos variabes de mayor peso de la validacion cruzada

In [ ]:
import pandas as pd
import numpy as np

# Obtener coeficientes absolutos del modelo ya entrenado
feature_importance = pd.Series(np.abs(logreg.coef_[0]), index=X.columns)

# Ordenar de mayor a menor importancia
top3_vars_cv = feature_importance.sort_values(ascending=False)

# Excluir 'tiempo_meses' si estuviera
top3_vars_cv = [var for var in top3_vars_cv.index if var != 'tiempo_meses'][:3]

print("Top 3 variables según regresión logística (validación cruzada):", top3_vars_cv)

In [ ]:
# Filtramos las columnas que no están en la lista "excluir"
df_filtrado = dfe.drop(columns=excluir)
df_filtrado.columns

#### Riesgos proporcionales de Cox (tres variables de mayor peso en Validación Cruzada)

In [ ]:
import statsmodels.formula.api as smf

# df para modelo de Cox con esas variables
df_cox = dfe[['tiempo_meses', 'Muerte_o_Infarto'] + top3_vars_cv].dropna()

# Paso 2: Modelo de Cox con esas variables
modelo_cox = smf.phreg(
    formula="tiempo_meses ~ " + " + ".join(top3_vars_cv),
    data=df_cox,
    status=df_cox["Muerte_o_Infarto"],
    ties="efron"
)

# Ajustamos modelo
resultado = modelo_cox.fit()
print(resultado.summary())

In [ ]:
# variable tipo_procedimiento (ACTP = 0, Pontaje Aortocoronario = 1)
print(dfe['tipo_procedimiento'].value_counts())

#### Validacion cruzada datos escalados + Youden (configurada con Random Forest)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, roc_curve
)
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar y preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte',
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []
overall_confusion_matrix = np.zeros((2, 2))

# Ejecutar validación cruzada
for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Modelo: Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predicciones probabilísticas
    y_pred_proba = rf.predict_proba(X_test)[:, 1]

    # Calcular índice de Youden para encontrar el mejor umbral
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    youden_index = tpr - fpr
    best_idx = np.argmax(youden_index)
    best_threshold = thresholds[best_idx]

    # Clasificación con umbral óptimo
    y_pred = (y_pred_proba >= best_threshold).astype(int)

    # Métricas
    cm = confusion_matrix(y_test, y_pred)
    overall_confusion_matrix += cm
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    print(f"Fold {fold}: Mejor umbral (Youden) = {best_threshold:.2f}")
    print(f"Fold {fold}: Sensibilidad = {recall:.2f}, Especificidad = {cm[0,0]/(cm[0,0]+cm[0,1]):.2f}, Accuracy = {accuracy:.2f}\n")

    fold_results.append({
        'Fold': fold,
        'Threshold': best_threshold,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1 Score': f1,
        'ROC-AUC': roc_auc
    })

# Resultados
results_df = pd.DataFrame(fold_results)
print("Resultados por Fold:")
print(results_df)

# Promedios
mean_results = results_df.mean(numeric_only=True)
print("\nPromedio de métricas:")
print(mean_results)

# Matriz de confusión total
print("\nMatriz de Confusión Total:")
print(overall_confusion_matrix)

# Visualización de métricas por fold
results_df[['Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC-AUC']].boxplot()
plt.title('Distribución de métricas por Fold (Random Forest + Youden)')
plt.ylabel('Score')
plt.grid(alpha=0.3)
plt.show()

#### Curvas ROC folds validacion cruzada + youden (configurada con Random Forest)

In [ ]:
from matplotlib.colors import to_rgba
from matplotlib import colormaps
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Cargar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte',
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Función para aclarar colores
def lighten_color(color, amount=0.5):
    r, g, b, a = to_rgba(color)
    return (r + (1 - r) * amount, g + (1 - g) * amount, b + (1 - b) * amount, a)

# Preparar colores
colors = colormaps['tab10']
colors_light = [lighten_color(colors(i / 9), amount=0.5) for i in range(5)]

# Variables para curva promedio
tprs = []
mean_fpr = np.linspace(0, 1, 100)

# Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Graficar curvas ROC por fold
plt.figure(figsize=(10, 6))
for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Entrenar modelo
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred_proba = rf.predict_proba(X_test)[:, 1]

    # Curva ROC y Youden
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    youden_index = tpr - fpr
    best_idx = np.argmax(youden_index)
    best_threshold = thresholds[best_idx]
    best_fpr = fpr[best_idx]
    best_tpr = tpr[best_idx]

    # Interpolación para curva media
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Dibujar curva ROC del fold
    plt.plot(fpr, tpr, color=colors_light[fold - 1],
             label=f'Fold {fold} (AUC = {roc_auc:.2f}, Threshold = {best_threshold:.2f})')
    
    # Punto óptimo
    plt.scatter(best_fpr, best_tpr, color=colors_light[fold - 1], edgecolors='black', zorder=5)

# Curva ROC media
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue', linewidth=2, label=f'Mean ROC (AUC = {mean_auc:.2f})')

# Línea de referencia
plt.plot([0, 1], [0, 1], linestyle='--', color='red')

# Estética
plt.title('Curvas ROC por Fold con Umbral Óptimo (Youden) - Random Forest', fontsize=16)
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### Extraemos variabes de mayor peso de la validacion cruzada + youden (configurada con Random Forest)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Cargar y preparar datos
excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte',
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']
X = dfe.drop(columns=excluir, errors='ignore')
y = dfe['Muerte_o_Infarto']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Acumulador de importancias
importancias_acumuladas = np.zeros(X.shape[1])

# Entrenar modelos y acumular importancias
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, y_train = X_scaled[train_idx], y.iloc[train_idx]
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    importancias_acumuladas += rf.feature_importances_

# Calcular importancia media
importancia_media = importancias_acumuladas / cv.get_n_splits()

# Crear Series con nombres de variables
feature_importance_rf = pd.Series(importancia_media, index=X.columns)

# Ordenar y seleccionar las 3 más importantes
top3_vars_cv_rf = feature_importance_rf.sort_values(ascending=False)

# Excluir 'tiempo_meses' si estuviera
top3_vars_cv_rf = [var for var in top3_vars_cv_rf.index if var != 'tiempo_meses'][:3]

print("Top 3 variables según Random Forest (validación cruzada):", top3_vars_cv_rf)

#### Riesgos proporcionales de Cox (tres variables de mayor peso en Validación Cruzada + Youden) (configurada con Random Forest)

In [ ]:
import statsmodels.formula.api as smf

# df para modelo de Cox con esas variables
df_cox_rf = dfe[['tiempo_meses', 'Muerte_o_Infarto'] + top3_vars_cv_rf].dropna()

# Paso 2: Modelo de Cox con esas variables
modelo_cox_rf = smf.phreg(
    formula="tiempo_meses ~ " + " + ".join(top3_vars_cv_rf),
    data=df_cox_rf,
    status=df_cox_rf["Muerte_o_Infarto"],
    ties="efron"
)

# Ajustamos modelo
resultado_rf = modelo_cox_rf.fit()
print(resultado_rf.summary())

In [ ]:
dfe.columns

#### Random Forest (Umbral optimo)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

excluir = ['Muerte_o_Infarto', 'NHC', 'ApellidosyNombre', 'Fecha_Intervención', 'fecha_alta', 'Fecha_muerte', 'Fecha_infarto', 
           'Fecha_Nueva_revascularización', 'Fecha_Ingreso_ICC', 'Causa', 'tipo_causa', 'Tipo_intervención_mitral', 'Muerte', 'Infarto', 'dias_muerte', 
           'Nueva_revascularización', 'Ingeso_ICC', 'DA_revasc', 'Cx_revasc', 'CD_revasc', 'Tabaquismo_Rec', 'dias_FEVI_post', 'tiempo_meses', 'FEVI_post', 'dias_FEVI_pre']

X = dfe.drop(columns=excluir, errors='ignore')  # Variables independientes
y = dfe['Muerte_o_Infarto']  # Variable dependiente

# Escalamos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividimos datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Entrenamos modelo Random Forest 
rf = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    max_depth=None, 
    min_samples_split=2, 
)
rf.fit(X_train, y_train)

# Probabilidades predichas
y_pred_proba = rf.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva

# Calculamos curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Determinamos umbral óptimo máximo de sens + esp
youden_index = tpr - fpr
optimal_idx = np.argmax(youden_index)
optimal_threshold = thresholds[optimal_idx]
print(f"Umbral óptimo (máximo sensibilidad + especificidad): {optimal_threshold:.2f}")

# Predicciones con umbral óptimo
y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)

# Matriz de confusión con umbral óptimo
cm = confusion_matrix(y_test, y_pred_optimal)
print("Matriz de Confusión (Umbral Óptimo):")
print(cm)

# sensibilidad, especificidad y precisión
tn, fp, fn, tp = cm.ravel()
sensibilidad = tp / (tp + fn)
especificidad = tn / (tn + fp)
exactitud = accuracy_score(y_test, y_pred_optimal)

print(f"\nExactitud: {exactitud:.2f}")
print(f"Sensibilidad (Recall): {sensibilidad:.2f}")
print(f"Especificidad: {especificidad:.2f}")

print("\nReporte de Clasificación (Umbral Óptimo):")
print(classification_report(y_test, y_pred_optimal, zero_division=0))

# Métricas adicionales
roc_auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred_optimal, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred_optimal, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred_optimal, average='macro', zero_division=0)

print(f"\nROC-AUC Score: {roc_auc:.2f}")
print(f"Precisión (macro): {precision:.2f}")
print(f"Recall (macro): {recall:.2f}")
print(f"F1-Score (macro): {f1:.2f}")

# Visualización de resultados
# Matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Evento', 'Evento'], yticklabels=['No Evento', 'Evento'])
plt.title('Matriz de Confusión (Umbral Óptimo)')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

# Curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})', color='blue')
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label=f'Optimal Threshold ({optimal_threshold:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='red')  # Línea de referencia
plt.title('Curva ROC')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Validación cruzada para evaluar el modelo
cv_scores = cross_val_score(rf, X_scaled, y, cv=5, scoring='accuracy')  # 5 particiones
print("Resultados de Validación Cruzada (Accuracy):", cv_scores)
print(f"Promedio de Validación Cruzada: {cv_scores.mean():.2f}")

##### Importancia de variables por reducción de impureza

In [ ]:
# Importancia de variables por reducción de impureza
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Recuperar nombres de variables originales
feature_names = X.columns

# Obtener importancias y ordenarlas
importancias = rf.feature_importances_
importancias_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False)

# Mostrar TODAS las variables en el gráfico
plt.figure(figsize=(10, len(importancias_df) * 0.3))  # Ajusta alto en función del número de variables
sns.barplot(x='Importancia', y='Variable', data=importancias_df, palette='viridis')
plt.title('Importancia de variables según reducción de impureza')
plt.xlabel('Importancia normalizada')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()

# (Opcional) Mostrar la tabla completa en pantalla
print("\nImportancia de todas las variables (ordenadas):")
print(importancias_df.reset_index(drop=True))

##### Importancia de variables por reducción de impureza (Top 15)

In [ ]:
# Importancia de variables por reducción de impureza
import pandas as pd

# Recuperar nombres de variables originales
feature_names = X.columns

# Obtener importancias y ordenarlas
importancias = rf.feature_importances_
importancias_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False)

# Mostrar top 15 más importantes (puedes ajustar a tus necesidades)
top_n = 15
top_importancias = importancias_df.head(top_n)

# Gráfico de barras horizontal
plt.figure(figsize=(10, 6))
sns.barplot(x='Importancia', y='Variable', data=top_importancias, palette='viridis')
plt.title(f'Importancia de variables según reducción de impureza (top {top_n})')
plt.xlabel('Importancia normalizada')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()

# (Opcional) Mostrar la tabla completa en pantalla
print("\nImportancia de todas las variables (ordenadas):")
print(importancias_df.reset_index(drop=True))

#### Gráfico Shap Values (Random Forest)

In [ ]:
import shap
import matplotlib.pyplot as plt

# Convertir X_test de nuevo a DataFrame con nombres de columnas
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Explicador SHAP basado en árboles
explainer = shap.TreeExplainer(rf, feature_perturbation="tree_path_dependent")

# Obtener valores SHAP
shap_values = explainer.shap_values(X_test_df)

# Si hay más de una clase, seleccionamos la clase positiva
if isinstance(shap_values, list):  
    shap_values = shap_values[1]  # Para clasificación binaria con lista
elif shap_values.ndim == 3:
    shap_values = shap_values[:, :, 1]  # Para clasificación binaria con array 3D

# Graficar SHAP Summary Plot con todas las variables
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_df, plot_type="dot", max_display=len(X_test_df.columns))

In [ ]:
import numpy as np

# Calcular importancia media absoluta de los SHAP values
shap_importance = np.abs(shap_values).mean(axis=0)

# Obtener índice ordenado (descendente) de importancia
sorted_idx = np.argsort(shap_importance)[::-1]

# Obtener las 4 variables más importantes
top4_vars = X_test_df.columns[sorted_idx[:4]].tolist()

# Eliminar 'tiempo_meses' si está presente
top3_vars_shap = [var for var in top4_vars if var != 'tiempo_meses'][:3]

print("Top 3 variables según SHAP values (excluyendo tiempo_meses):")
print(top3_vars_shap)

In [ ]:
'DA_sin_revasc'

#### Riesgos proporcionales de Cox (tres variables de mayor peso en Shap values)

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Asegúrate de que no haya valores nulos en las variables seleccionadas
variables_shap = ['FEVI_pre', 'edad_ingreso', 'HTA']
variables_modelo = ['tiempo_meses', 'Muerte_o_Infarto'] + variables_shap

# Filtrar y eliminar filas con datos faltantes
df_cox_shap = dfe[variables_modelo].dropna()

# Definir la fórmula del modelo de Cox
formula_cox = "tiempo_meses ~ " + " + ".join(variables_shap)

# Ajustar el modelo de riesgos proporcionales de Cox
modelo_cox_shap = smf.phreg(
    formula=formula_cox,
    data=df_cox_shap,
    status=df_cox_shap["Muerte_o_Infarto"],  # Variable de evento (1=evento, 0=censura)
    ties="efron"  # Método para manejo de empates (recomendado)
)

# Entrenar el modelo
resultado_shap = modelo_cox_shap.fit()

# Mostrar resumen de resultados
print(resultado_shap.summary())